# Load libraries

In [2]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.path.dirname('src'), '..')))

import pandas as pd
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from catboost import Pool
from src.tools import *

import warnings
warnings.filterwarnings("ignore")

In [3]:
def check_result(model,X_test,y_test):
  metrics_dict = {}
  fpr, tpr, thresholds = metrics.roc_curve(y_test, model.predict(X_test))
  auc_score = metrics.auc(fpr, tpr)
  f1_sc = metrics.f1_score(y_test, model.predict(X_test), average='macro')
  metrics_dict['auc_score'] = auc_score
  metrics_dict['f1_score'] = f1_sc
  return metrics_dict

In [4]:
def search_num_features(df, feature_importance, upsamp_func: Optional[Literal['ROS', 'RUS', 'SMOTE', 'OSS']] = None, p = 0.3, step = 5):
  best_score = 0
  best_num_features = 0

  for num_col in tqdm(range(1, len(feature_importance), step)):
    features = list(feature_importance.iloc[:num_col,:]['feature_names'])
    data = df[features + ['Machine failure']]
    X_train, X_test, y_train, y_test = train_test_split(data.drop(columns = ['Machine failure']),
                                                        data['Machine failure'],
                                                        test_size=0.33,
                                                        random_state=42,
                                                        stratify = df['Machine failure'])
    
    if upsamp_func != 'ROS':
      X_train, y_train = DataSampler.ROS(X_train, y_train, p)
    elif upsamp_func != 'RUS':
      X_train, y_train = DataSampler.RUS(X_train, y_train, p)
    elif upsamp_func != 'SMOTE':
      X_train, y_train = DataSampler.SMOTE(X_train, y_train, p)
    elif upsamp_func != 'OSS':
      X_train, y_train = DataSampler.OSS(X_train, y_train)
    
    train_pool = Pool(data=X_train, label=y_train)
    CatBoost = CatBoostClassifier(verbose=False,random_seed=42)
    CatBoost.fit(train_pool)
    metrics_dict = check_result(CatBoost, X_test, y_test)
    print(f'F1_score - {metrics_dict["f1_score"]}, num_features - {best_num_features}, AUC_score = {metrics_dict["auc_score"]}')
    if metrics_dict['f1_score'] > best_score:
      best_score = metrics_dict['f1_score']
      best_num_features = num_col
  print(f'Best F1_score - {best_score}, num_features - {best_num_features}')


# Read data

In [5]:
df = pd.read_csv('../data/prepared_data.csv')
df_fe = pd.read_csv('../data/data_fe.csv')

# Train Catboost

## Test data without upsampling

### Find feature importance

In [5]:
CatBoost = CatBoostClassifier(verbose=False,random_seed=42)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(df_fe.drop(columns = ['Machine failure']),
                                                    df_fe['Machine failure'],
                                                    test_size=0.33,
                                                    random_state=42,
                                                    stratify = df['Machine failure'])

train_pool = Pool(data=X_train, label=y_train)

In [7]:
CatBoost.fit(train_pool)

feature_importance = pd.DataFrame({'feature_importance': CatBoost.get_feature_importance(train_pool), 
              'feature_names': X_test.columns}).sort_values(by=['feature_importance'], 
                                                       ascending=False)
              
check_result(CatBoost, X_test, y_test )

{'auc_score': 0.9003741709983868, 'f1_score': 0.9241142395957508}

In [8]:
feature_importance.to_csv('../data/feature_imporstance/feature_imporstance_WS.csv', index = False)

### Find best amount of features

In [9]:
search_num_features(df_fe,feature_importance, step = 5)

  0%|          | 0/25 [00:00<?, ?it/s]

F1_score - 0.6734304006608839, num_features - 0, AUC_score = 0.6151303997132103


  4%|▍         | 1/25 [00:04<01:50,  4.61s/it]

F1_score - 0.7779974810761374, num_features - 1, AUC_score = 0.7291629324251659


  8%|▊         | 2/25 [00:09<01:46,  4.62s/it]

...

F1_score - 0.9329922027290448, num_features - 36, AUC_score = 0.9180744757124933


 96%|█████████▌| 24/25 [02:47<00:09,  9.29s/it]

F1_score - 0.9296546462113844, num_features - 36, AUC_score = 0.9093027424269583


100%|██████████| 25/25 [02:57<00:00,  9.38s/it]


Best F1_score - 0.9345060499209441, num_features - 36


## Test data with upsampling

### Find feature importance

In [10]:
CatBoost = CatBoostClassifier(verbose=False,random_seed=42)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(df_fe.drop(columns = ['Machine failure']),
                                                    df_fe['Machine failure'],
                                                    test_size=0.33,
                                                    random_state=42,
                                                    stratify = df['Machine failure'])
X_train, y_train = upsampling(1,X_train, y_train)

train_pool = Pool(data=X_train, label=y_train)

In [12]:
CatBoost.fit(train_pool)

feature_importance = pd.DataFrame({'feature_importance': CatBoost.get_feature_importance(train_pool), 
              'feature_names': X_test.columns}).sort_values(by=['feature_importance'], 
                                                       ascending=False)
              
check_result(CatBoost, X_test, y_test)

{'auc_score': 0.9225387614267789, 'f1_score': 0.9356730254831562}

In [13]:
feature_importance.to_csv('../data/feature_imporstance/feature_imporstance.csv',index = False)

### Find best amount of features

In [14]:
search_num_features(df_fe, feature_importance, upsamp_func = True, step = 5)

  0%|          | 0/25 [00:00<?, ?it/s]

F1_score - 0.5030190363209421, num_features - 0, AUC_score = 0.7000358487184083


  4%|▍         | 1/25 [00:06<02:25,  6.08s/it]

F1_score - 0.8971506553910363, num_features - 1, AUC_score = 0.9194019985660513


  8%|▊         | 2/25 [00:12<02:23,  6.22s/it]

...

F1_score - 0.9366441795347409, num_features - 111, AUC_score = 0.9140807044273167


 96%|█████████▌| 24/25 [04:09<00:13, 13.66s/it]

F1_score - 0.9399027517255195, num_features - 111, AUC_score = 0.9228524377128516


100%|██████████| 25/25 [04:24<00:00, 13.99s/it]


Best F1_score - 0.9399027517255195, num_features - 111
